In [3]:
import numpy as np
from MCTS import MCTS
from Game import *
from model import AmoebaZeroModel

In [4]:
#test_probs = np.ones( (N,N) )
#test_probs /= NN


test_probs = np.ones( (N,N) )
for x in range(N):
    for y in range(N):
        test_probs[x,y] = 1.0 / (abs(N/2-x) + abs(N/2-y) + 1.0)**3
test_probs /= sum(sum(test_probs))

def evaluator(g):
    return test_probs, 0.5


In [5]:
g = Game()
nr=0
while (g.winner==0):
    if nr%2==0:
        x = np.random.randint(N)
        y = np.random.randint(N)
        while g.grid[x,y]!=0:
            x = np.random.randint(N)
            y = np.random.randint(N)
        g.move(x,y)
        g.print()
    else:
        mcts = MCTS(g, evaluator)
        x,y,_ = mcts.select_move()
        g.move(x,y)
        g.print() 
    nr+=1
print(nr, g.winner)


 ABCDEFGHIJKLMNO
-----------------
|X              |1
|               |2
|               |3
|               |4
|               |5
|               |6
|               |7
|               |8
|               |9
|               |10
|               |11
|               |12
|               |13
|               |14
|               |15
-----------------
 ABCDEFGHIJKLMNO
-----------------
|X              |1
|               |2
|               |3
|               |4
|               |5
|               |6
|               |7
|               |8
|          O    |9
|               |10
|               |11
|               |12
|               |13
|               |14
|               |15
-----------------
 ABCDEFGHIJKLMNO
-----------------
|X              |1
|X              |2
|               |3
|               |4
|               |5
|               |6
|               |7
|               |8
|          O    |9
|               |10
|               |11
|               |12
|               |13
|               |14
|     

 ABCDEFGHIJKLMNO
-----------------
|X              |1
|X       X      |2
|               |3
|               |4
|   O X         |5
|               |6
|       XO  X   |7
|     O  OO     |8
|      OO  O    |9
|    O X XO     |10
|     X   O     |11
|          XX  X|12
|               |13
|         X     |14
|   O           |15
-----------------
 ABCDEFGHIJKLMNO
-----------------
|X              |1
|X       X      |2
|               |3
|               |4
|   O X         |5
|         O     |6
|       XO  X   |7
|     O  OO     |8
|      OO  O    |9
|    O X XO     |10
|     X   O     |11
|          XX  X|12
|               |13
|         X     |14
|   O           |15
-----------------
 ABCDEFGHIJKLMNO
-----------------
|X              |1
|X       X      |2
|               |3
|               |4
|   O X         |5
|         O     |6
|       XO  X   |7
|     O  OO     |8
|      OO  O    |9
|    O X XO     |10
|     X   O     |11
|          XX  X|12
|X              |13
|         X     |14
|   O 

In [6]:
g = Game()
n = None
nr = 0
while (g.winner==0):
    mcts = MCTS(g, evaluator, n)
    x,y,n = mcts.select_move()
    g.move(x,y)
    g.print() 
    nr+=1
print(nr,g.winner)


 ABCDEFGHIJKLMNO
-----------------
|               |1
|               |2
|               |3
|               |4
|               |5
|               |6
|               |7
|               |8
|        X      |9
|               |10
|               |11
|               |12
|               |13
|               |14
|               |15
-----------------
 ABCDEFGHIJKLMNO
-----------------
|               |1
|               |2
|               |3
|               |4
|               |5
|               |6
|               |7
|       O       |8
|        X      |9
|               |10
|               |11
|               |12
|               |13
|               |14
|               |15
-----------------
 ABCDEFGHIJKLMNO
-----------------
|               |1
|               |2
|               |3
|               |4
|               |5
|               |6
|               |7
|       O       |8
|        X    X |9
|               |10
|               |11
|               |12
|               |13
|               |14
|     

 ABCDEFGHIJKLMNO
-----------------
|               |1
|               |2
|       O       |3
|            O  |4
|       XX      |5
|       O X     |6
|      XX       |7
|     XOOXOO    |8
|      OXX  O X |9
|       XO X    |10
|        O      |11
|               |12
|               |13
|   O           |14
|             X |15
-----------------
 ABCDEFGHIJKLMNO
-----------------
|               |1
|               |2
|       O       |3
|            O  |4
| O     XX      |5
|       O X     |6
|      XX       |7
|     XOOXOO    |8
|      OXX  O X |9
|       XO X    |10
|        O      |11
|               |12
|               |13
|   O           |14
|             X |15
-----------------
 ABCDEFGHIJKLMNO
-----------------
|               |1
|               |2
|       O       |3
|            O  |4
| O     XX      |5
|       O X     |6
|      XX       |7
|    XXOOXOO    |8
|      OXX  O X |9
|       XO X    |10
|        O      |11
|               |12
|               |13
|   O           |14
|     

 ABCDEFGHIJKLMNO
-----------------
|               |1
|         X     |2
|       OX      |3
|        XX  O  |4
| O     XX      |5
|      XO X     |6
|      XXOX     |7
|    XXOOXOOO   |8
|  O  OOXXOXO X |9
|      OXO XX   |10
|     O OOO     |11
|        OX     |12
|            X  |13
|   O   O       |14
|           X X |15
-----------------
 ABCDEFGHIJKLMNO
-----------------
|               |1
|         X     |2
|       OX      |3
|        XX  O  |4
| O     XX      |5
|      XO X     |6
|      XXOX     |7
|    XXOOXOOO   |8
|  O  OOXXOXO X |9
| O    OXO XX   |10
|     O OOO     |11
|        OX     |12
|            X  |13
|   O   O       |14
|           X X |15
-----------------
 ABCDEFGHIJKLMNO
-----------------
|               |1
|         X     |2
|       OX      |3
|        XX  O  |4
| O     XX      |5
|      XO X     |6
|      XXOX     |7
|    XXOOXOOO   |8
|  O  OOXXOXO X |9
| O    OXOXXX   |10
|     O OOO     |11
|        OX     |12
|            X  |13
|   O   O       |14
|     

In [7]:
m = AmoebaZeroModel()

def evaluator(g):
    return m.evaluate(g)
    


In [8]:
g = Game()
nr=0
while (g.winner==0):
    if nr%2==0:
        x = np.random.randint(N)
        y = np.random.randint(N)
        while g.grid[x,y]!=0:
            x = np.random.randint(N)
            y = np.random.randint(N)
        g.move(x,y)
        g.print()
    else:
        mcts = MCTS(g, evaluator)
        x,y,_ = mcts.select_move()
        g.move(x,y)
        g.print() 
    nr+=1
print(nr, g.winner)


 ABCDEFGHIJKLMNO
-----------------
|               |1
|               |2
|               |3
|               |4
|               |5
|               |6
|               |7
|       X       |8
|               |9
|               |10
|               |11
|               |12
|               |13
|               |14
|               |15
-----------------
 ABCDEFGHIJKLMNO
-----------------
|               |1
|               |2
|               |3
|               |4
|               |5
|               |6
|               |7
|       X    O  |8
|               |9
|               |10
|               |11
|               |12
|               |13
|               |14
|               |15
-----------------
 ABCDEFGHIJKLMNO
-----------------
|               |1
|               |2
|               |3
|               |4
|      X        |5
|               |6
|               |7
|       X    O  |8
|               |9
|               |10
|               |11
|               |12
|               |13
|               |14
|     

 ABCDEFGHIJKLMNO
-----------------
|        O  O   |1
|       X O     |2
|               |3
| O   X    X    |4
|      X X  X   |5
|          O  X |6
|            O  |7
|       X   OOX |8
|  O            |9
|   O       X   |10
| X             |11
|         X     |12
|        X O    |13
|      O    O   |14
|               |15
-----------------
 ABCDEFGHIJKLMNO
-----------------
|        O  O   |1
|       X O     |2
|               |3
| O  XX    X    |4
|      X X  X   |5
|          O  X |6
|            O  |7
|       X   OOX |8
|  O            |9
|   O       X   |10
| X             |11
|         X     |12
|        X O    |13
|      O    O   |14
|               |15
-----------------
 ABCDEFGHIJKLMNO
-----------------
|        O  O   |1
|       X O     |2
|               |3
| O  XX    X    |4
|      X X  X   |5
|          O  X |6
|            O  |7
|       X   OOX |8
|  O            |9
|   O       X   |10
| X             |11
|         X     |12
|O       X O    |13
|      O    O   |14
|     

 ABCDEFGHIJKLMNO
-----------------
|   X    O  O   |1
|       X O     |2
|      XO   O O |3
| O  XXO   X    |4
|      X X  X   |5
|  O      XO XX |6
| O          O  |7
|  O    X   OOX |8
| XO          X |9
|   O       X   |10
| X  X       X  |11
|O       OX   O |12
|O    O  X O  X |13
|      O    O   |14
|    X         X|15
-----------------
 ABCDEFGHIJKLMNO
-----------------
|   X    O  O   |1
|       X O     |2
|      XO   O O |3
| O  XXO   X    |4
|    X X X  X   |5
|  O      XO XX |6
| O          O  |7
|  O    X   OOX |8
| XO          X |9
|   O       X   |10
| X  X       X  |11
|O       OX   O |12
|O    O  X O  X |13
|      O    O   |14
|    X         X|15
-----------------
 ABCDEFGHIJKLMNO
-----------------
|   X    O  O   |1
|       X O     |2
|      XO   O O |3
| O  XXO   X    |4
|    X X X  X   |5
|  O      XO XX |6
| O          O  |7
|  O    X   OOX |8
| XO       O  X |9
|   O       X   |10
| X  X       X  |11
|O       OX   O |12
|O    O  X O  X |13
|      O    O   |14
|    X

 ABCDEFGHIJKLMNO
-----------------
|   X    O  O   |1
|    O  X O     |2
|     OXO   O O |3
| O  XXO   X X  |4
|    X X X  X   |5
|  O X X  XO XX |6
| OX    O  O O  |7
| OO   OX   OOX |8
| XO O  X  OX X |9
|   O    O  X   |10
| X  X       XOX|11
|O     X OXO  O |12
|O    O OX O  X |13
|X     O X  O   |14
|    XX        X|15
-----------------
 ABCDEFGHIJKLMNO
-----------------
|   X    O  O   |1
|    OX X O     |2
|     OXO   O O |3
| O  XXO   X X  |4
|    X X X  X   |5
|  O X X  XO XX |6
| OX    O  O O  |7
| OO   OX   OOX |8
| XO O  X  OX X |9
|   O    O  X   |10
| X  X       XOX|11
|O     X OXO  O |12
|O    O OX O  X |13
|X     O X  O   |14
|    XX        X|15
-----------------
 ABCDEFGHIJKLMNO
-----------------
|   X    O  O   |1
|    OX X O     |2
|     OXO   O O |3
| O  XXO   X X  |4
|    X X X  X   |5
|  O X X  XO XX |6
| OX    O  O O  |7
| OO   OX   OOX |8
| XO O  X  OX X |9
|   O    O  X   |10
| X  X    O  XOX|11
|O     X OXO  O |12
|O    O OX O  X |13
|X     O X  O   |14
|    X

In [9]:
g = Game()
nr=0
while (g.winner==0):
    if nr%2==0:
        x = np.random.randint(N)
        y = np.random.randint(N)
        while g.grid[x,y]!=0:
            x = np.random.randint(N)
            y = np.random.randint(N)
        g.move(x,y)
        g.print()
    else:
        mcts = MCTS(g, evaluator)
        x,y,_ = mcts.select_move()
        g.move(x,y)
        g.print() 
    nr+=1
print(nr, g.winner)


 ABCDEFGHIJKLMNO
-----------------
|               |1
|               |2
|               |3
|               |4
|               |5
|               |6
|               |7
|               |8
|               |9
|      X        |10
|               |11
|               |12
|               |13
|               |14
|               |15
-----------------
 ABCDEFGHIJKLMNO
-----------------
|               |1
|               |2
|               |3
|               |4
|               |5
|               |6
|               |7
|               |8
|          O    |9
|      X        |10
|               |11
|               |12
|               |13
|               |14
|               |15
-----------------
 ABCDEFGHIJKLMNO
-----------------
|               |1
|               |2
|               |3
|               |4
|               |5
|               |6
|               |7
|               |8
|  X       O    |9
|      X        |10
|               |11
|               |12
|               |13
|               |14
|     

 ABCDEFGHIJKLMNO
-----------------
|      X        |1
| O      X      |2
|      X        |3
|        XO  X  |4
|O   X        O |5
|   O         X |6
|   X          X|7
|              O|8
|  X       O    |9
| O    X      O |10
|               |11
| O O O         |12
|   X           |13
|   X O         |14
|               |15
-----------------
 ABCDEFGHIJKLMNO
-----------------
|      X        |1
| O      X      |2
|      X        |3
|        XO  X  |4
|O   X        O |5
|   O         X |6
|   X          X|7
|    X         O|8
|  X       O    |9
| O    X      O |10
|               |11
| O O O         |12
|   X           |13
|   X O         |14
|               |15
-----------------
 ABCDEFGHIJKLMNO
-----------------
|      X        |1
| O      X      |2
|      X        |3
|        XO  X  |4
|O   X        O |5
|   O         X |6
|   X          X|7
|    X         O|8
|  X       O    |9
| O    X      O |10
|               |11
| O O O         |12
|   X           |13
|   X O         |14
|     

 ABCDEFGHIJKLMNO
-----------------
|   O  X        |1
| O      X O    |2
|      X    O   |3
|        XO  X  |4
|O   X    O   O |5
|  OO         X |6
|   X          X|7
|    X  X      O|8
|  XX    X O    |9
| O    X O XO O |10
|    X     OO   |11
| O O O         |12
| O X           |13
|  XXOO     X   |14
|   X OXX   X   |15
-----------------
 ABCDEFGHIJKLMNO
-----------------
|   O  X        |1
| O      X O    |2
|      X    O   |3
|        XO  X  |4
|O   X    O   O |5
|  OO         X |6
|   X          X|7
|    X  X      O|8
|  XXX   X O    |9
| O    X O XO O |10
|    X     OO   |11
| O O O         |12
| O X           |13
|  XXOO     X   |14
|   X OXX   X   |15
-----------------
 ABCDEFGHIJKLMNO
-----------------
|   O  X        |1
| O      X O    |2
|      X    O   |3
|        XO  X  |4
|O   X    O   O |5
|  OO         X |6
|   X          X|7
|    X  X      O|8
|  XXX   X O    |9
| O    X O XO O |10
|    X     OO   |11
| O O O         |12
| O X  O        |13
|  XXOO     X   |14
|   X 

 ABCDEFGHIJKLMNO
-----------------
|   O XX        |1
| O      X O   O|2
|  XO  X   XOO  |3
| X      XO  X  |4
|O   XOXO O   O |5
|  OO    X  O XO|6
|   X         XX|7
| X  X OX      O|8
|  XXX XOX O    |9
| O   XX O XO O |10
|    X O   OO   |11
| O O O         |12
|OO X  O        |13
|  XXOO     X   |14
|   XXOXX   X   |15
-----------------
 ABCDEFGHIJKLMNO
-----------------
|   O XX        |1
| O      X OX  O|2
|  XO  X   XOO  |3
| X      XO  X  |4
|O   XOXO O   O |5
|  OO    X  O XO|6
|   X         XX|7
| X  X OX      O|8
|  XXX XOX O    |9
| O   XX O XO O |10
|    X O   OO   |11
| O O O         |12
|OO X  O        |13
|  XXOO     X   |14
|   XXOXX   X   |15
-----------------
 ABCDEFGHIJKLMNO
-----------------
|   O XX        |1
| O      X OX  O|2
|  XO  X   XOO  |3
| X      XO  X  |4
|O   XOXO O   O |5
|  OO    X  O XO|6
|   X         XX|7
| X  X OX      O|8
|  XXX XOX O    |9
| O   XX O XO O |10
|    X OO  OO   |11
| O O O         |12
|OO X  O        |13
|  XXOO     X   |14
|   XX

In [10]:
p = np.zeros( (1,NN) )

g = Game()
mcts = MCTS(g, evaluator)
_,_,_ = mcts.select_move()
mcts.fill_p(p)
print(p)

[[  0.00000000e+00   0.00000000e+00   0.00000000e+00   3.69443299e-06
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    3.69443299e-06   0.00000000e+00   0.00000000e+00   3.69443299e-06
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00  

In [11]:
g.move(8,8)
mcts = MCTS(g, evaluator)
_,_,_ = mcts.select_move()
mcts.fill_p(p)
print(p)

[[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   5.03285759e-03
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   1.45577749e-01   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00  